In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
path = '../PostBornEma/'
sys.path.append("../scripts/") 
from lab import *

### 1) Terms that also appear in Limber approximation

#### Collecting all contributions from Eq.s 4.2-4.5 that are also non-zero in Limber

Term 1:
$- \int_{\vec{l}} [\vec{L}\cdot \vec{l}] [\vec{l}\cdot (\vec{L}-\vec{l})]^2 \int \mathrm{d}\chi \int \mathrm{d}\chi' W(\chi,\chi_{CMB}) W^h(\chi') C_l^{\delta\psi}(\chi',\chi) C_{|\vec{L}-\vec{l}|}^{\phi\phi}(\chi',\chi) = - \int_{\vec{l}} [\vec{L}\cdot \vec{l}] [\vec{l}\cdot (\vec{L}-\vec{l})]^2 M^{A}(l,|\vec{L}-\vec{l}|)$
after  transform $\vec{l} \rightarrow \vec{l}+\vec{L}$

$\int_{\vec{l}} [\vec{L}\cdot  (\vec{L}+\vec{l})] [\vec{l}\cdot (\vec{L}+\vec{l})]^2 M^{A}( |\vec{L}+\vec{l}|,l)$

Term 2: 
$ L^2 \int_{\vec{l}} [\vec{L}\cdot \vec{l}]^2 \int \mathrm{d}\chi \int \mathrm{d}\chi' W(\chi,\chi_{CMB}) W^h(\chi') C_L^{\delta\psi}(\chi',\chi) [\frac{1}{2}C_{l}^{\phi\phi}(\chi')+\frac{1}{2}C_{l}^{\phi\phi}(\chi)]$
$ = L^2 \int_{\vec{l}} [\vec{L}\cdot \vec{l}]^2 \tilde{M}^{A}(L,l)$

Comparison with Limber results suggest that we are missing a minus sign in Term 2. Will be coding with this Minus sign

Adding Term 1 and 2 with trick from Krause & Hirata:
$\int_{\vec{l}} [\vec{L}\cdot  (\vec{L}+\vec{l})] [\vec{l}\cdot (\vec{L}+\vec{l})]^2 [M^{A}( |\vec{L}+\vec{l}|,l)-\tilde{M}^{A}(L,l)] +  \left[\int_{\vec{l}} [\vec{L}\cdot  (\vec{L}+\vec{l})] [\vec{l}\cdot (\vec{L}+\vec{l})]^2-L^2[\vec{L}\cdot \vec{l}]^2\right] \tilde{M}^{A}(L,l)$


In [14]:
# naming corresponds to order of axes
#clpsidelta = np.load('../G_matrices/clphidelta.npy')
clphidelta = np.load('../G_matrices/clphidelta.npy')
clphiphi   = np.load('../G_matrices/clphiphi_parallel.npy')
clphipsi   = np.load('../G_matrices/clphipsi.npy')
print(clpd.shape)

(289, 100, 100)


In [15]:
#clphipsi*clphiphi
clprod1 = np.zeros((len(ell_),len(ell_),len(t_),len(t_)))
for ii in range(len(ell_)):
    for jj in range(len(ell_)):
        clprod1[ii][jj] = clphipsi[ii]*clphiphi[jj]
        
  

In [16]:
# clpd*clpp, first axis is chi' second is chi 
clprod2 = np.zeros((len(ell_),len(ell_),len(t_),len(t_)))
for ii in range(len(ell_)):
    for jj in range(len(ell_)):
        clprod2[ii][jj] = clphidelta[ii]*clphiphi[jj] 

In [10]:
#getting MA
bin_num = 'all'

def lensing_kernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi)

#r2d goes with phi,phi
#t2d goes with delta/psi
r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)

kernel2 = gal_clus(dNdz_LSST,constant_bias,bin_num)
res = np.sum(clprod2*lensing_kernel(r2d*chi_cmb,chi_cmb)*kernel2(t2d*chi_cmb)*t2d*chi_cmb*w11*w12, axis =(2,3))
res.shape

(289, 289)

array([0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014314,
       0.00014314, 0.00014314, 0.00014314, 0.00014314, 0.00014